# M&A Report 

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
from datetime import datetime


# ==========================
# 셀 1 — 파일 목록 & 비교 대상 자동 선택
# ==========================

# SY 회사 drayage 폴더
DATA_DIR = Path("data") / "SY" / "drayage"

# 폴더 내 엑셀 파일 수집
files = list(DATA_DIR.glob("*.xlsx"))


# 파일명에서 날짜(앞 8자리: MMDDYYYY) 추출 → datetime.date 변환
def extract_date(path: Path):
    """
    파일명 예: 11202025 - 2.xlsx → 11/20/2025
    """
    m = re.match(r"(\d{8})", path.name)
    if not m:
        return None
    raw = m.group(1)
    try:
        return datetime.strptime(raw, "%m%d%Y").date()
    except ValueError:
        return None


# 날짜 기준 정렬 (오류 또는 없는 날짜는 최솟값 처리)
files_sorted = sorted(
    files,
    key=lambda p: extract_date(p) or datetime.min.date()
)

# 최신 2개 선정
prev_file = files_sorted[-2]      # 이전날 리포트
curr_file = files_sorted[-1]      # 최신 리포트

prev_date = extract_date(prev_file)
curr_date = extract_date(curr_file)

today = datetime.today().date()

print(f"- Today        : {today}")
print(f"- 최신 리포트     : {curr_date}  ({curr_file.name})")
print(f"- 이전 리포트     : {prev_date}  ({prev_file.name})")

- Today        : 2025-11-23
- 최신 리포트     : 2025-11-21  (11212025 - 2.xlsx)
- 이전 리포트     : 2025-11-20  (11202025 - 2.xlsx)


In [2]:
# ==========================
# 셀 2 — 전날/오늘 리포트 로드 + 공통 컬럼 정리
# ==========================

KEY = ["Customer Reference No.", "Container No."]

DISPLAY_COLS = [
    "Customer Reference No.",
    "Pick Up Location",
    "Master B/L No.",
    "Container No.",
    "Last Free Date",
    "P/U APPT DATE",
    "P/U APPT TIME",
    "EMPTY NOTICE",
    "Container Remark",
    "DELIVERY DATE",
    "Return Date",
]

# 1) 파일 읽기
df_prev = pd.read_excel(prev_file)
df_curr = pd.read_excel(curr_file)

# 2) 컬럼 공백/개행 제거
df_prev.columns = df_prev.columns.str.strip()
df_curr.columns = df_curr.columns.str.strip()

# 3) 필요 컬럼 추가 (없는 경우 NaN 생성)
for col in DISPLAY_COLS:
    if col not in df_prev.columns:
        df_prev[col] = pd.NA
    if col not in df_curr.columns:
        df_curr[col] = pd.NA

# 4) DISPLAY_COLS 순서대로 통일
df_prev = df_prev[DISPLAY_COLS]
df_curr = df_curr[DISPLAY_COLS]

print("셀2 완료 — 리포트 로드 및 컬럼 정리 완료")


셀2 완료 — 리포트 로드 및 컬럼 정리 완료


---

## 일자별 요약 리포트

In [3]:
# ==========================
# 셀 3 — 전날/오늘 리포트 변경 사항 요약
#  - 어떤 컬럼이든 바뀐 컨테이너 행 전부 카운트 (셀4와 동일 기준)
#  - 날짜별 변화 + 배송 예정일(DEL / DEL-D) + 예외(SUPA DUPA URGENT 등) 요약
#  - MT 로 시작하는 Remark 는 예외 카운트에서도 제외
# ==========================

import re

# 1. 인덱스를 Customer Reference No. + Container No. 로 설정
df_prev_k = df_prev.set_index(KEY)
df_curr_k = df_curr.set_index(KEY)

prev_idx = df_prev_k.index
curr_idx = df_curr_k.index

# 2. 공통 컨테이너 / 오늘만 있는 컨테이너
common_idx    = prev_idx.intersection(curr_idx)
curr_only_idx = curr_idx.difference(prev_idx)

# 3. 공통 컨테이너만 사용해서 비교 준비
prev_c = df_prev_k.loc[common_idx]
curr_c = df_curr_k.loc[common_idx]

shared_cols = [c for c in prev_c.columns if c in curr_c.columns]
prev_c2 = prev_c[shared_cols]
curr_c2 = curr_c[shared_cols]

# 4. 셀 단위 비교 (NaN 동일 처리)
equal_mask   = (prev_c2 == curr_c2) | (prev_c2.isna() & curr_c2.isna())
changed_mask = ~equal_mask

# 5. 주요 컬럼 정의 (셀4 기준과 동일)
important_cols = [
    "Last Free Date",
    "P/U APPT DATE",
    "P/U APPT TIME",
    "Container Remark",   # 배송 예정 텍스트
    "DELIVERY DATE",
    "Return Date",
]
important_cols = [c for c in important_cols if c in shared_cols]

# 🔹 셀4와 동일한 기준으로 "변경된 컨테이너 행" 계산
row_changed_mask   = changed_mask[important_cols]
changed_rows_count = int(row_changed_mask.any(axis=1).sum())

# 7. 신규 컨테이너 수
added_containers = len(curr_only_idx)

# 8. 주요 컬럼별 변경/신규 통계
column_stats = []
for col in important_cols:
    prev_col = prev_c2[col]
    curr_col = curr_c2[col]

    new_values    = int((prev_col.isna() & curr_col.notna()).sum())
    changed_cells = int(changed_mask[col].sum())

    column_stats.append({
        "Column": col,
        "New": new_values,
        "Changed_cells": changed_cells,
    })

# 9. 한글 컬럼명
col_kor = {
    "Last Free Date":   "LFD",
    "P/U APPT DATE":    "픽업 날짜",
    "P/U APPT TIME":    "픽업 시간",
    "Container Remark": "배송 예정일(Delivery)",
    "DELIVERY DATE":    "배송 완료일(Delivered)",
    "Return Date":      "리턴 날짜(Return)",
    "Delivery":         "배송 예정일(Delivery)",
}

# ==========================
# 10. 날짜별 변화 요약 (LFD / 픽업 / 배송 완료 / 리턴)
# ==========================
date_change_stats = {}

for col in ["Last Free Date", "P/U APPT DATE", "DELIVERY DATE", "Return Date"]:
    if col not in shared_cols:
        continue

    prev_dates = pd.to_datetime(prev_c2[col], errors="coerce").dt.date.fillna("Empty")
    curr_dates = pd.to_datetime(curr_c2[col], errors="coerce").dt.date.fillna("Empty")

    prev_count = prev_dates.value_counts()
    curr_count = curr_dates.value_counts()

    all_keys = sorted(set(prev_count.index).union(curr_count.index), key=str)

    detail = []
    total_change = 0

    for k in all_keys:
        if k == "Empty":
            continue

        prev_n = int(prev_count.get(k, 0))
        curr_n = int(curr_count.get(k, 0))

        if prev_n != curr_n:
            diff = curr_n - prev_n
            sign = "+" if diff > 0 else ""
            total_change += abs(diff)
            detail.append(f"  - {k}: {prev_n} 건 → {curr_n} 건 ({sign}{diff} 건)")

    date_change_stats[col] = {"total": total_change, "details": detail}

# ==========================
# 11. 배송 예정일(Delivery) 파싱 + 집계
#  - DEL / DEL DIRECT 를 날짜별로 합침
#  - 'OR' 날짜 2개 이상 → 예외
#  - 날짜 없는 텍스트(SUPA DUPA URGENT 등) → 예외
#  - MT 로 시작하는 텍스트는 예외에서도 제외
# ==========================

def parse_delivery(text):
    """
    반환: (date, tag, is_multi)
    - 정상 케이스: (날짜, 'DEL' 또는 'DEL-D', False)
    - 복수 날짜: (None, None, True)
    - 나머지(단일 날짜 없음): (None, None, False)
    """
    if not isinstance(text, str):
        return (None, None, False)

    low = text.lower()
    if "del" not in low:
        return (None, None, False)

    matches = re.findall(r"(\d{1,2})/(\d{1,2})", text)
    if len(matches) == 0:
        return (None, None, False)
    if len(matches) > 1:
        # 날짜 2개 이상 → 예외로 처리
        return (None, None, True)

    mm, dd = matches[0]
    year = curr_date.year if "curr_date" in globals() and curr_date else 2025
    dt = pd.to_datetime(f"{year}-{mm}-{dd}", errors="coerce")

    tag = "DEL-D" if "direct" in low else "DEL"
    return (dt.date(), tag, False)


if "Container Remark" in shared_cols:
    prev_remark = prev_c2["Container Remark"].fillna("")
    curr_remark = curr_c2["Container Remark"].fillna("")

    prev_parsed = prev_remark.apply(parse_delivery)
    curr_parsed = curr_remark.apply(parse_delivery)
else:
    prev_remark = pd.Series([""] * len(prev_c2), index=prev_c2.index)
    curr_remark = pd.Series([""] * len(curr_c2), index=curr_c2.index)
    prev_parsed = pd.Series([(None, None, False)] * len(prev_c2), index=prev_c2.index)
    curr_parsed = pd.Series([(None, None, False)] * len(curr_c2), index=curr_c2.index)

prev_date_d  = prev_parsed.apply(lambda x: x[0])
prev_type_d  = prev_parsed.apply(lambda x: x[1])
prev_multi_d = prev_parsed.apply(lambda x: x[2])

curr_date_d  = curr_parsed.apply(lambda x: x[0])
curr_type_d  = curr_parsed.apply(lambda x: x[1])
curr_multi_d = curr_parsed.apply(lambda x: x[2])

# 정상 단일 날짜 DEL/DEL-D만
prev_valid = prev_date_d.notna() & (~prev_multi_d)
curr_valid = curr_date_d.notna() & (~curr_multi_d)

_prev_dates = prev_date_d[prev_valid]
_prev_types = prev_type_d[prev_valid]

_curr_dates = curr_date_d[curr_valid]
_curr_types = curr_type_d[curr_valid]

prev_total = _prev_dates.value_counts()
curr_total = _curr_dates.value_counts()

_prev_df = pd.DataFrame({"date": _prev_dates, "type": _prev_types})
_curr_df = pd.DataFrame({"date": _curr_dates, "type": _curr_types})

prev_type_cnt = _prev_df.groupby(["date", "type"]).size() if len(_prev_df) > 0 else pd.Series(dtype="int64")
curr_type_cnt = _curr_df.groupby(["date", "type"]).size() if len(_curr_df) > 0 else pd.Series(dtype="int64")

all_del_dates = sorted(set(prev_total.index).union(curr_total.index))

delivery_lines = []
delivery_total_change = 0

for d in all_del_dates:
    prev_n = int(prev_total.get(d, 0))
    curr_n = int(curr_total.get(d, 0))
    if prev_n != curr_n:
        diff = curr_n - prev_n
        sign = "+" if diff > 0 else ""
        delivery_total_change += abs(diff)

        p_del  = int(prev_type_cnt.get((d, "DEL"),   0))
        p_deld = int(prev_type_cnt.get((d, "DEL-D"), 0))
        c_del  = int(curr_type_cnt.get((d, "DEL"),   0))
        c_deld = int(curr_type_cnt.get((d, "DEL-D"), 0))

        delivery_lines.append(
            f"  - {d}: {prev_n} 건 → {curr_n} 건 ({sign}{diff} 건) "
            f"(DEL: {p_del} → {c_del}, DEL-D: {p_deld} → {c_deld})"
        )

# ==========================
# 12. 배송일정 예외 (텍스트 기반):
#   - 'DEL 11/22 OR 11/24' 같은 복수 날짜
#   - 'SUPA DUPA URGENT' 같은 날짜 없는 텍스트
#   - MT 로 시작하는 텍스트는 예외에서도 제외
# ==========================

def is_delivery_exception(text: str) -> bool:
    if not isinstance(text, str):
        return False
    t = text.strip()
    if not t:
        return False

    up = t.upper()

    # ✅ MT 로 시작하는 건 전부 제외 (예외 카운트 안 함)
    if up.startswith("MT"):
        return False

    # 복수 날짜 → 예외
    d, tag, is_multi = parse_delivery(t)
    if is_multi:
        return True

    # 정상 단일 날짜 DEL/DEL-D → 예외 아님
    if d is not None:
        return False

    # 여기까지 오면 날짜 없는 텍스트 → 예외 (예: SUPA DUPA URGENT)
    return True

prev_exc_mask  = prev_remark.apply(is_delivery_exception)
curr_exc_mask  = curr_remark.apply(is_delivery_exception)

prev_exc_texts = prev_remark[prev_exc_mask]
curr_exc_texts = curr_remark[curr_exc_mask]

prev_exc_counts = prev_exc_texts.value_counts()
curr_exc_counts = curr_exc_texts.value_counts()
all_exc_texts   = sorted(set(prev_exc_counts.index).union(curr_exc_counts.index))

# ==========================
# 13. 출력
# ==========================

print("=== SY Drayage 리포트 변경 요약 ===")
print(f"- 신규 컨테이너 추가: {added_containers} 건")
print(f"- 값이 변경된 컨테이너 행: {changed_rows_count} 건")

for item in column_stats:
    name = col_kor.get(item["Column"], item["Column"])
    print(f"- {name}: 변경 {item['Changed_cells']} 건 (New: {item['New']} 건)")

print("\n--- 날짜별 변화 요약 ---")

# 1) LFD
lfd = date_change_stats["Last Free Date"]
print(f"[LFD] 총 {lfd['total']} 건 변경")
for l in lfd["details"]:
    print(l)

# 2) 픽업 날짜
pu = date_change_stats["P/U APPT DATE"]
print(f"[픽업 날짜] 총 {pu['total']} 건 변경")
for l in pu["details"]:
    print(l)

# 3) 배송 예정일(Delivery) — 날짜가 확실한 DEL / DEL-D
print(f"[배송 예정일(Delivery)] 총 {delivery_total_change} 건 변경")
for line in delivery_lines:
    print(line)

# 3-1) 배송일정 예외 — 텍스트 그대로 + 이전/오늘 건수 (MT*는 제외됨)
if len(all_exc_texts) > 0:
    print("배송일정 예외:")
    for txt in all_exc_texts:
        prev_n = int(prev_exc_counts.get(txt, 0))
        curr_n = int(curr_exc_counts.get(txt, 0))
        print(f"  '{txt}': 이전 {prev_n} 건, 오늘 {curr_n} 건")

# 4) 배송 완료일(Delivered)
deliv = date_change_stats["DELIVERY DATE"]
print(f"[배송 완료일(Delivered)] 총 {deliv['total']} 건 변경")
for l in deliv["details"]:
    print(l)

# 5) 리턴 날짜(Return)
ret = date_change_stats["Return Date"]
print(f"[리턴 날짜(Return)] 총 {ret['total']} 건 변경")
for l in ret["details"]:
    print(l)


=== SY Drayage 리포트 변경 요약 ===
- 신규 컨테이너 추가: 0 건
- 값이 변경된 컨테이너 행: 65 건
- LFD: 변경 4 건 (New: 4 건)
- 픽업 날짜: 변경 18 건 (New: 2 건)
- 픽업 시간: 변경 16 건 (New: 2 건)
- 배송 예정일(Delivery): 변경 56 건 (New: 39 건)
- 배송 완료일(Delivered): 변경 18 건 (New: 18 건)
- 리턴 날짜(Return): 변경 17 건 (New: 17 건)

--- 날짜별 변화 요약 ---
[LFD] 총 4 건 변경
  - 2025-11-25: 0 건 → 4 건 (+4 건)
[픽업 날짜] 총 7 건 변경
  - 2025-11-20: 23 건 → 20 건 (-3 건)
  - 2025-11-21: 13 건 → 15 건 (+2 건)
  - 2025-11-22: 0 건 → 2 건 (+2 건)
[배송 예정일(Delivery)] 총 31 건 변경
  - 2025-11-21: 15 건 → 0 건 (-15 건) (DEL: 15 → 0, DEL-D: 0 → 0)
  - 2025-11-22: 0 건 → 12 건 (+12 건) (DEL: 0 → 12, DEL-D: 0 → 0)
  - 2025-11-24: 9 건 → 8 건 (-1 건) (DEL: 1 → 0, DEL-D: 8 → 8)
  - 2025-11-25: 3 건 → 6 건 (+3 건) (DEL: 3 → 6, DEL-D: 0 → 0)
배송일정 예외:
  'DEL 11/22 OR 11/24': 이전 0 건, 오늘 8 건
  'LINE HOLD ERROR': 이전 1 건, 오늘 0 건
  'SUPA DUPA URGENT': 이전 0 건, 오늘 1 건
[배송 완료일(Delivered)] 총 18 건 변경
  - 2025-11-21: 0 건 → 18 건 (+18 건)
[리턴 날짜(Return)] 총 17 건 변경
  - 2025-11-21: 0 건 → 17 건 (+17 건)


---

## Final Report 

In [4]:
# ==========================
# 셀 4 — 변경된 컨테이너 상세 표
# ==========================

import numpy as np
import re
from datetime import datetime

# --- 1. 인덱스 기반 준비 ---
df_prev_k = df_prev.set_index(KEY)
df_curr_k = df_curr.set_index(KEY)

prev_idx = df_prev_k.index
curr_idx = df_curr_k.index

common_idx = prev_idx.intersection(curr_idx)

prev_c = df_prev_k.loc[common_idx]
curr_c = df_curr_k.loc[common_idx]

shared_cols = [c for c in prev_c.columns if c in curr_c.columns]
prev_c2 = prev_c[shared_cols]
curr_c2 = curr_c[shared_cols]

equal_mask = (prev_c2 == curr_c2) | (prev_c2.isna() & curr_c2.isna())
changed_mask = ~equal_mask

row_cols_for_change = [
    "Last Free Date",
    "P/U APPT DATE",
    "P/U APPT TIME",
    "Container Remark",
    "DELIVERY DATE",
    "Return Date",
]
row_cols_for_change = [c for c in row_cols_for_change if c in shared_cols]

row_changed = changed_mask[row_cols_for_change].any(axis=1)

changed_curr = df_curr_k.loc[row_changed].copy()


# --- 2. Delivery 생성 ---
def format_delivery(text: str):
    if not isinstance(text, str):
        return ("", None)

    t = text.strip()
    if not t:
        return ("", None)

    up = t.upper()

    # MT* → 숨김
    if up.startswith("MT"):
        return ("", None)

    # 날짜 찾기
    matches = re.findall(r"(\d{1,2})/(\d{1,2})", t)
    if len(matches) == 0:
        return (t, None)

    if len(matches) > 1:
        return (t, None)

    mm, dd = matches[0]
    year = curr_date.year
    full_date = f"{year}-{int(mm):02d}-{int(dd):02d}"

    tag = "DEL-D" if "DIRECT" in up else "DEL"

    try:
        s_date = datetime.strptime(full_date, "%Y-%m-%d").date()
    except:
        s_date = None

    return (f"{full_date} ({tag})", s_date)


delivery_display, delivery_sort = zip(
    *changed_curr["Container Remark"].apply(format_delivery)
)
changed_curr["Delivery"] = list(delivery_display)
changed_curr["Delivery_sort"] = list(delivery_sort)


# --- 3. 날짜 & 시간 처리 ---
for col in ["Last Free Date", "P/U APPT DATE", "DELIVERY DATE", "Return Date"]:
    changed_curr[col] = pd.to_datetime(changed_curr[col], errors="coerce").dt.date

changed_curr["P/U APPT TIME"] = pd.to_datetime(
    changed_curr["P/U APPT TIME"], errors="coerce"
).dt.strftime("%H:%M")


changed_curr = changed_curr.replace("NaT", "").fillna("")


# --- 4. 정렬: LFD → Delivery_sort ---
changed_curr = changed_curr.sort_values(
    by=["Last Free Date", "Delivery_sort"],
    ascending=[True, True],
    na_position="last"
)

# --- 5. 최종 컬럼 순서 확정 ---
changed_curr = changed_curr.reset_index()

rename_map = {
    "Customer Reference No.": "PO#",
    "Master B/L No.": "MBL#",
    "Container No.": "CNTR#",
    "Last Free Date": "LFD",
    "P/U APPT DATE": "PU Date",
    "P/U APPT TIME": "PU Time",
    "DELIVERY DATE": "Delivered",
    "Return Date": "Return",
}

changed_curr = changed_curr.rename(columns=rename_map)

FINAL_COLS = ["PO#", "MBL#", "CNTR#", "LFD", "PU Date", "PU Time",
              "Delivery", "Delivered", "Return"]

changed_table = changed_curr[FINAL_COLS].copy()

# --- 6. index 1부터 ---
changed_table.index = range(1, len(changed_table) + 1)


# --- 7. 하이라이트 ---
display_to_orig = {
    "LFD": "Last Free Date",
    "PU Date": "P/U APPT DATE",
    "PU Time": "P/U APPT TIME",
    "Delivery": "Container Remark",
    "Delivered": "DELIVERY DATE",
    "Return": "Return Date",
}

def highlight(row):
    styles = []
    po, cntr = row["PO#"], row["CNTR#"]
    key = (po, cntr)
    prev_row = df_prev_k.loc[key]
    curr_row = df_curr_k.loc[key]

    for col in row.index:
        if col not in display_to_orig:
            styles.append("")
            continue

        orig = display_to_orig[col]
        prev_val, curr_val = prev_row.get(orig), curr_row.get(orig)
        same = ((pd.isna(prev_val) and pd.isna(curr_val)) or prev_val == curr_val)

        if same:
            styles.append("")
        else:
            styles.append("background-color: #fff3b0; color: red;")

    return styles


styler = (
    changed_table.style
        .set_table_styles([{"selector": "th", "props":[("text-align","center")]}])
        .apply(highlight, axis=1)
)

print(f"변경된 컨테이너 수: {len(changed_table)} 건\n")
styler


변경된 컨테이너 수: 65 건



,PO#,MBL#,CNTR#,LFD,PU Date,PU Time,Delivery,Delivered,Return
1,25-6051,SMLMSEL5K8966200,SMCU1229899,2025-11-14,2025-11-14,23:00,,2025-11-17,2025-11-18
2,25-6053,SMLMSEL5K8202600,SMCU1163209,2025-11-14,2025-11-14,23:00,,2025-11-17,2025-11-18
3,25-5954,KORPPNC4658070,TEMU8979328,2025-11-17,2025-11-17,07:00,,2025-11-17,2025-11-21
4,25-6870,KORPPNC4651058,KMTU9354810,2025-11-17,2025-11-17,09:00,,2025-11-17,2025-11-21
5,25-6828,ZIMUSEL71188158,JXLU4660830,2025-11-18,2025-11-21,11:00,DEL 11/22 OR 11/24,,
6,25-6836,ZIMUSEL71188161,BEAU6334358,2025-11-18,2025-11-18,14:00,,2025-11-19,2025-11-21
7,25-6841,ZIMUSEL71188163,CAAU9554975,2025-11-18,2025-11-18,14:00,,2025-11-19,2025-11-21
8,25-6842,ZIMUSEL71188151,ZCSU7184234,2025-11-18,2025-11-18,15:00,,2025-11-19,2025-11-21
9,25-6843,ZIMUSEL71188143,TGBU7130207,2025-11-18,2025-11-18,14:00,,2025-11-20,2025-11-21
10,25-6829,ZIMUSEL71188186,JXLU6326186,2025-11-19,2025-11-21,10:00,2025-11-22 (DEL),,


---

## Save

In [7]:
# ==========================
# 셀 5 — 정렬해서 엑셀 파일로 저장
#  - 정렬: 1) LFD, 2) Return 오름차순
# ==========================

from pathlib import Path
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font, Alignment

OUTPUT_DIR = Path("output")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

date_str = curr_date.strftime("%Y%m%d")
output_path = OUTPUT_DIR / f"sy_drayage_diff_{date_str}.xlsx"

# 1) 저장용 DataFrame: LFD → Return 기준으로 정렬
sort_cols = [c for c in ["LFD", "Return"] if c in changed_table.columns]
df_to_save = (
    changed_table
    .sort_values(by=sort_cols, ascending=True, na_position="last")
    .copy()
)

# 2) 정렬된 df_to_save 를 그대로 엑셀로 저장
df_to_save.to_excel(output_path, index=True, sheet_name="SY_Drayage_Diff")

# 3) 스타일 적용
wb = load_workbook(output_path)
ws = wb.active

max_col = ws.max_column
for col in range(1, max_col + 1):
    cell = ws.cell(row=1, column=col)
    cell.alignment = Alignment(horizontal="center", vertical="center")

fill_changed = PatternFill(fill_type="solid", fgColor="FFF3B0")
font_changed = Font(color="FF0000")

# df_to_save 기준으로 다시 돌면서 변경 셀 체크
for excel_row_idx, (df_idx, row) in enumerate(df_to_save.iterrows(), start=2):
    po   = row["PO#"]
    cntr = row["CNTR#"]
    key = (po, cntr)

    try:
        prev_row = df_prev_k.loc[key]
        curr_row = df_curr_k.loc[key]
    except KeyError:
        continue

    for col_offset, col_name in enumerate(df_to_save.columns, start=2):  # 2열부터 데이터
        if col_name not in display_to_orig:   # LFD, PU Date, Delivery, Delivered, Return 등만
            continue

        orig_col = display_to_orig[col_name]
        prev_val = prev_row.get(orig_col, pd.NA)
        curr_val = curr_row.get(orig_col, pd.NA)

        same = (
            (pd.isna(prev_val) and pd.isna(curr_val)) or
            (prev_val == curr_val)
        )

        if not same:
            cell = ws.cell(row=excel_row_idx, column=col_offset)
            cell.fill = fill_changed
            cell.font = font_changed

# 열 너비 자동 조정
for column_cells in ws.columns:
    max_length = 0
    col_letter = column_cells[0].column_letter
    for cell in column_cells:
        value = cell.value
        if value is None:
            continue
        value_str = str(value)
        if len(value_str) > max_length:
            max_length = len(value_str)
    ws.column_dimensions[col_letter].width = max_length + 2

wb.save(output_path)
print(f"정렬까지 완료해서 저장: {output_path}")


정렬까지 완료해서 저장: output/sy_drayage_diff_20251121.xlsx
